# 1st EOF in the tropical Pacific for FOSI

In [ ]:
### GENERAL SETUP
%matplotlib inline  
# this enables plotting within notebook

#import modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
import numpy as np   # basic math library  you will type np.$STUFF  e.g., np.cos(1)
import numpy.linalg as LA
from matplotlib.gridspec import GridSpec
import timeit
import cartopy.crs as ccrs
import datetime
import scipy.stats as stats # imports stats functions https://docs.scipy.org/doc/scipy/reference/stats.html
import cartopy.feature as cfeature

In [ ]:
# from @GlacialMeg

import matplotlib as mpl
# Font style and size
plt.rcParams['font.family'] = 'Arial'         # Font
plt.rcParams['font.size'] = 10                # General font size unless set below
plt.rcParams['axes.labelsize'] = 11           # Axes labels font size
plt.rcParams['figure.titlesize'] = 12         # Title font size
plt.rcParams['figure.titleweight'] = 'bold'   # Bold title
plt.rcParams['axes.labelweight'] = 'bold'     # Bold axes labels
    
# Axes and ticks parameters
plt.rcParams['axes.linewidth'] = 1            # Width of axes border
plt.rcParams['xtick.direction'] = 'in'        # Make x ticks go in
plt.rcParams['ytick.direction'] = 'in'        # Make y ticks go in
plt.rcParams['xtick.major.size'] = 5          # Set x tick length 
plt.rcParams['ytick.major.size'] = 5          # Set y tick length
plt.rcParams['xtick.major.width'] = 1         # Set x tick width 
plt.rcParams['ytick.major.width'] = 1         # Set y tick width

# Line style
plt.rcParams['lines.linewidth'] = 1           # Set line widths on plots
plt.rcParams['lines.linestyle'] = '-'         # Set line styles on plots

# Math text font characteristics
plt.rcParams['mathtext.fontset'] = 'cm'       # Choose font for math text
plt.rcParams['mathtext.default'] = 'regular'  # Make math text not bold or italic
# mathtext.FontConstantsBase.sup1 = 0.4         # Move superscript text to a better height

# For showing plots on GitHub
%matplotlib inline
plt.rcParams['figure.dpi']= 100

In [ ]:
# wd = '/glade/work/smogen/SMYLE-extremes/notebooks/1.0.process.smyle/eof/'
wd = '/glade/work/smogen/SMYLE-extremes/notebooks/1.0.process.smyle/eof/use/'

In [ ]:
# Nino3.4
ds = xr.open_dataset('/glade/work/smogen/SMYLE-extremes/FOSI/TEMP.monthly.surface.regrid.nc')['TEMP']
ds['time'] = pd.date_range("1958-01", "2020-12", freq="MS")
ds = ds.sel(lat=slice(-5,5),lon=slice(210 - 360,270 - 360))
ds = ds.weighted(np.cos(np.deg2rad(ds.lat))).mean(dim=('lat','lon'))
ds = ds.groupby('time.month') - ds.groupby('time.month').mean()

## Compare PCs

In [ ]:
var = 'TEMP'
pc_temp = xr.open_dataset(wd + var + '.principal_components.final.nc')['__xarray_dataarray_variable__']

var = 'omega'
pc_omega = xr.open_dataset(wd + var + '.principal_components.final.nc')['__xarray_dataarray_variable__']

var = 'omega_residual'
pc_omegares = xr.open_dataset(wd + var + '.principal_components.final.nc')['__xarray_dataarray_variable__']

var = 'H'
pc_ph = xr.open_dataset(wd + var + '.principal_components.final.nc')['__xarray_dataarray_variable__']

var = 'H_residual'
pc_phres = xr.open_dataset(wd + var + '.principal_components.final.nc')['__xarray_dataarray_variable__']

var = 'H_temp'
pc_phtemp = xr.open_dataset(wd + var + '.principal_components.final.nc')['__xarray_dataarray_variable__']

var = 'SSH'
pc_ssh = xr.open_dataset(wd + var + '.principal_components.final.nc')#['__xarray_dataarray_variable__']
pc_ssh = pc_ssh.rename({'__xarray_dataarray_variable__':'SSH'}); pc_ssh =pc_ssh.SSH

In [ ]:
f, ax = plt.subplots(1,1,figsize=(10,5))
# pc_temp.isel(number=0).plot(color='red',label='SST PC1',linewidth=0.8)
# pc_omega.isel(number=0).plot(color='blue',label='$\Omega_{arag}$ PC1',linewidth=0.8)
# pc_ssh.isel(number=0).plot(color='k',label='SSH PC1',linewidth=0.8)

pc_phres.isel(number=0).plot(color='navy',label='$H^+$ residual PC1',linewidth=0.8,linestyle='-.')
pc_phtemp.isel(number=0).plot(color='navy',label='$H^+$ temperature PC1',linewidth=0.8,linestyle='--')
pc_ph.isel(number=0).plot(color='navy',label='$H^+$ original PC1',linewidth=0.8)

ds.plot(color='grey',label='Nino3.4',linestyle='--',linewidth=1)
plt.legend(loc='upper left')

plt.legend(loc='upper right')

xr.corr(pc_ssh.isel(number=0),ds)

In [ ]:
print('SSH and PC1 of Omega r=' +str(np.round(xr.corr(pc_ssh.isel(number=0),pc_omega.isel(number=0)).values,3)))
print('SSH and PC1 of pH_residual r=' +str(np.round(xr.corr(pc_ssh.isel(number=0),pc_phres.isel(number=0)).values,3)))
print('PC1 of pH_residual and PC1 of Omega r=' +str(np.round(xr.corr(pc_phres.isel(number=0),pc_omega.isel(number=0)).values,3)))
print('PC1 of pH and PC1 of Omega r=' +str(np.round(xr.corr(pc_ph.isel(number=0),pc_omega.isel(number=0)).values,3)))
print('PC1 of pH and PC1 of pH residual r=' +str(np.round(xr.corr(pc_ph.isel(number=0),pc_phres.isel(number=0)).values,3)))

## Compare EOFs
we know that the first EOF is basically ENSO - how correlated are they with eachother though?

In [ ]:
var = 'TEMP'
eof_temp = xr.open_dataset(wd + var + '.EOFs.final.nc')#['__xarray_dataarray_variable__']
eof_temp = eof_temp.rename({'__xarray_dataarray_variable__':'SST'})
eof_temp = eof_temp.SST

var = 'omega'
eof_omega = xr.open_dataset(wd + var + '.EOFs.final.nc')#['__xarray_dataarray_variable__']
eof_omega = eof_omega.rename({'__xarray_dataarray_variable__':'$\Omega_{arag}$'})
eof_omega =eof_omega['$\Omega_{arag}$']

var = 'omega_residual'
eof_omegares = xr.open_dataset(wd + var + '.EOFs.final.nc')#['__xarray_dataarray_variable__']
eof_omegares = eof_omegares.rename({'__xarray_dataarray_variable__':'omega residual'})
eof_omegares = eof_omegares['omega residual']

var = 'H'
eof_ph = xr.open_dataset(wd + var + '.EOFs.final.nc')#['__xarray_dataarray_variable__']
eof_ph = eof_ph.rename({'__xarray_dataarray_variable__':'pH'})
eof_ph = eof_ph.pH

var = 'H_residual'
eof_phres = xr.open_dataset(wd + var + '.EOFs.final.nc')#['__xarray_dataarray_variable__']
eof_phres = eof_phres.rename({'__xarray_dataarray_variable__':'pH residual'})
eof_phres = eof_phres['pH residual']

var = 'SSH'
eof_ssh = xr.open_dataset(wd + var + '.EOFs.final.nc')#['__xarray_dataarray_variable__']
eof_ssh = eof_ssh.rename({'__xarray_dataarray_variable__':'SSH'}); eof_ssh =eof_ssh.SSH

In [ ]:
f, ax = plt.subplots(5,1,figsize=(7,10),subplot_kw=dict(projection=ccrs.PlateCarree(central_longitude=180)))

eof_temp.isel(number=0).plot(ax = ax[0],cmap='coolwarm',transform = ccrs.PlateCarree())
ax[0].set_title('EOF1: SST')
ax[0].add_feature(cfeature.LAND, color='k', zorder=3)

eof_ssh.isel(number=0).plot(ax = ax[1],transform = ccrs.PlateCarree(),cmap='coolwarm')
ax[1].set_title('EOF1: SSH')
ax[1].add_feature(cfeature.LAND, color='k', zorder=3)

eof_omega.isel(number=0).plot(ax = ax[2],transform = ccrs.PlateCarree(),cmap='coolwarm')
ax[2].set_title('EOF1: $\Omega_{arag}$')
ax[2].add_feature(cfeature.LAND, color='k', zorder=3)

eof_omegares.isel(number=0).plot(ax = ax[3],transform = ccrs.PlateCarree(),cmap='coolwarm')
ax[3].set_title('EOF1: pH Temperature residual')
ax[3].add_feature(cfeature.LAND, color='k', zorder=3)


eof_phtemp.isel(number=0).plot(ax = ax[4],transform = ccrs.PlateCarree(),cmap='coolwarm')
ax[4].set_title('EOF1: pH Temperature')
ax[4].add_feature(cfeature.LAND, color='k', zorder=3)

## Paper Figure Draft

In [ ]:
f, ax = plt.subplots(1,1,figsize = (20,3))

ds.plot(color='k',label='Nino3.4 Index',linewidth=3,linestyle='--')

(-pc_temp).isel(number=0).plot(color='red',label='SST PC1',linewidth=2)
# pc_ssh.isel(number=0).plot(color='#7570b3',label='SSH PC1',linewidth=1)
(-pc_omega).isel(number=0).plot(color='cornflowerblue',label='$\Omega_{arag}$ PC1',linewidth=2)
(-pc_omegares).isel(number=0).plot(color='cornflowerblue',label='$\Omega_{arag}$ PC1',linewidth=2,linestyle='--')

(-pc_ph).isel(number=0).plot(color='blueviolet',label='pH PC1',linewidth=2)
(-pc_phres).isel(number=0).plot(color='blueviolet',label='pH PC1',linewidth=2,linestyle='--')

plt.legend(fontsize=14)
plt.title('')
plt.xlim('1958','2021')

plt.ylabel('Anomaly')

# f.savefig('./figures/Figure5.PC1.R1.pdf')
# f.savefig('./figures/Figure5.PC1.R1.eps')

In [ ]:
f, ax = plt.subplots(1,1,figsize = (10,2))

ds.plot(color='k',label='Nino3.4 Index',linewidth=3,linestyle='--')

(-pc_temp).isel(number=0).plot(color='red',label='SST PC1',linewidth=2)
# pc_ssh.isel(number=0).plot(color='#7570b3',label='SSH PC1',linewidth=1)
(-pc_omega).isel(number=0).plot(color='cornflowerblue',label='$\Omega_{arag}$ PC1',linewidth=2)
(-pc_omegares).isel(number=0).plot(color='darkblue',label='$\Omega_{arag}$ nonthermal PC1',linewidth=2,linestyle='--')

# (-pc_ph).isel(number=0).plot(color='blueviolet',label='pH PC1',linewidth=2)
# (-pc_phres).isel(number=0).plot(color='blueviolet',label='pH PC1',linewidth=2,linestyle='--')

# plt.legend(fontsize=14)
plt.title('')
plt.xlim('1958','2021')
plt.ylim(-3.8,3.8)

plt.ylabel('Anomaly')

f.savefig('./figures/Figure5.PC1.R1.omega.new.pdf')
# f.savefig('./figures/Figure5.PC1.R1.omega.eps')

In [ ]:
f, ax = plt.subplots(1,1,figsize = (10,2))

ds.plot(color='k',label='Nino3.4 Index',linewidth=3,linestyle='--')

# (-pc_temp).isel(number=0).plot(color='red',label='SST PC1',linewidth=2)
# pc_ssh.isel(number=0).plot(color='#7570b3',label='SSH PC1',linewidth=1)
# (-pc_omega).isel(number=0).plot(color='cornflowerblue',label='$\Omega_{arag}$ PC1',linewidth=2)
# (-pc_omegares).isel(number=0).plot(color='darkblue',label='$\Omega_{arag}$ PC1',linewidth=2,linestyle='--')

(-pc_ph).isel(number=0).plot(color='blueviolet',label='[$H^+$] PC1',linewidth=2)
(-pc_phres).isel(number=0).plot(color='purple',label='[$H^+$] nonthermal PC1',linewidth=2,linestyle='--')
(-pc_phtemp).isel(number=0).plot(color='red',label='[$H^+$] thermal PC1',linewidth=2,linestyle='--')
# (-(pc_phres + pc_phtemp).isel(number=0).plot(color='grey',label='[$H^+$] comb PC1',linewidth=2,linestyle='--')

# plt.legend(fontsize=14,loc='upper left')
plt.title('')
plt.xlim('1958','2021')
plt.ylim(-3.9,3.9)
plt.ylabel('Anomaly')

f.savefig('./figures/Figure5.PC1.R1.H.new.pdf')
# f.savefig('./figures/Figure5.PC1.R1.H.eps')

In [ ]:
f, ax = plt.subplots(1,5,figsize=(28,3),subplot_kw=dict(projection=ccrs.PlateCarree(central_longitude=180)))

(-eof_temp).isel(number=0).plot(ax = ax[0],cmap='coolwarm',transform = ccrs.PlateCarree(),extend='both',add_colorbar=False)
# ax[0].set_title('EOF1: SST')
ax[0].add_feature(cfeature.LAND, color='k', zorder=3)

# eof_ssh.isel(number=0).plot(ax = ax[1,0],transform = ccrs.PlateCarree(),cmap='coolwarm')
# ax[1,0].set_title('EOF1: SSH')
# ax[1,0].add_feature(cfeature.LAND, color='k', zorder=3)

(-eof_omega).isel(number=0).plot(ax = ax[1],transform = ccrs.PlateCarree(),cmap='coolwarm',levels=np.arange(-0.16,0.161,0.01),extend='both',add_colorbar=False)
# ax[1].set_title('EOF1: $\Omega_{arag}$')
ax[1].add_feature(cfeature.LAND, color='k', zorder=3)

(-eof_omegares).isel(number=0).plot(ax = ax[2],transform = ccrs.PlateCarree(),cmap='coolwarm',levels=np.arange(-0.16,0.161,0.01),extend='both',add_colorbar=False)
# ax[2].set_title('EOF1: $\Omega_{arag}$')
ax[2].add_feature(cfeature.LAND, color='k', zorder=3)

(-eof_ph).isel(number=0).plot(ax = ax[3],transform = ccrs.PlateCarree(),cmap='coolwarm',levels=np.arange(-4e-10,4.01e-10,1e-12),extend='both',add_colorbar=False)
# ax[3].set_title('EOF1: [H+]')
ax[3].add_feature(cfeature.LAND, color='k', zorder=3)

(-eof_phres).isel(number=0).plot(ax = ax[4],transform = ccrs.PlateCarree(),cmap='coolwarm',levels=np.arange(-4e-10,4.01e-10,1e-12),extend='both',add_colorbar=False)
# ax[4].set_title('EOF1: [H+] residual')
ax[4].add_feature(cfeature.LAND, color='k', zorder=3)

plt.subplots_adjust(wspace=0.15)
# plt.tight_layout()

f.savefig('./figures/Figure5.EOF1.R1.pdf',transparent=True)
# f.savefig('./figures/Figure5.EOF1.R1.eps')

In [ ]:
f, ax = plt.subplots(1,1,figsize = (20,3))

ds.plot(color='k',label='Nino3.4 Index',linewidth=3,linestyle='--')

# (-pc_temp).isel(number=0).plot(color='red',label='SST PC1',linewidth=2)
# pc_ssh.isel(number=0).plot(color='#7570b3',label='SSH PC1',linewidth=1)
# (-pc_omega).isel(number=0).plot(color='cornflowerblue',label='$\Omega_{arag}$ PC1',linewidth=2)
(pc_ph).isel(number=0).plot(color='blueviolet',label='[H+] PC1',linewidth=2)
(pc_phres).isel(number=0).plot(color='blueviolet',label='[H+] residual PC1',linewidth=2,linestyle='--')
(pc_phtemp).isel(number=0).plot(color='blueviolet',label='[H+] temp PC1',linewidth=2,linestyle='dotted')

plt.legend(fontsize=14)
plt.title('')
plt.xlim('1958','2021')

plt.ylabel('Anomaly')

# f.savefig('./figures/Figure5.PC1.pdf')
# f.savefig('./figures/Figure5.PC1.eps')

In [ ]:
import xskillscore as xs

In [ ]:
print(xr.corr(ds, pc_temp.isel(number=0)).values)
print(xr.corr(ds, pc_omega.isel(number=0)).values)
print(xr.corr(ds, pc_omegares.isel(number=0)).values)
print(xr.corr(ds, pc_ph.isel(number=0)).values)
print(xr.corr(ds, pc_phres.isel(number=0)).values)

In [ ]:
print([xs.pearson_r_eff_p_value(ds, pc_temp.isel(number=0)).values < 0.05])
print([xs.pearson_r_eff_p_value(ds, pc_omega.isel(number=0)).values < 0.05])
print([xs.pearson_r_eff_p_value(ds, pc_ph.isel(number=0)).values < 0.05])
print([xs.pearson_r_eff_p_value(ds, pc_phres.isel(number=0)).values < 0.05])

In [ ]:
print(xr.corr(ds, pc_ph.isel(number=0).shift(time=-9)).values)
print([xs.pearson_r_eff_p_value(ds, pc_ph.isel(number=0).shift(time=-9)).values < 0.05])

In [ ]:
xs.pearson_r_eff_p_value(ds, pc_ph.isel(number=0)).values